# Práctica 2

Procesamiento de Lenguaje Natural
Facultad de Ingeniería, UNAM

González Flores Andrés

## Instrucciones

A partir del corpus proporcionado ("corpusML.txt") realizar un modelo del lenguaje neuronal con base en la arquitectura propuesta por Bengio (2003).

Síganse los siguientes pasos:

1. Limpiar los textos y aplicar stemming a las palabras.
2. Insertar símbolos de inicio y final de cadena.
3. Obtener los bigramas que aparecen en el texto (indexar numéricamente).
4. Entrenar con los bigramas la red neuronal y obtener los valores para los hiperparámetros. Tomar de 100 unidades para la primera capa oculta (capa lineal) y 300 para la segunda capa oculta (capa con tanh).
5. Obtener las matrices $A$ y $\Pi$ a partir de las salidas de la red neuornal (probabilidad Softmax).
6. Evaluar el modelo (con Entropía).
7. Calcular la proabilidad de las siguientes oraciones:
    - Nos bañamos con agua caliente
    - El animalito le olía la cabeza
    - Pascuala ordeñaba las vacas

## Desarrollo

In [2]:
# Importar módulos
import re
from nltk import SnowballStemmer
from collections import Counter
from itertools import chain

In [3]:
# Definición de constantes
stemmer_esp = SnowballStemmer('spanish')
CORPUS_PATH = './Data/corpusML.txt'

### Paso 1

Limpiar los textos y aplicar stemming a las palabras.

In [4]:
with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
    corpus = [
        list(map(lambda token: stemmer_esp.stem(token), 
            re.findall('[a-zA-zñáéíóúü]+', frase.lower())
        ))
        for frase in f
    ]
    for stems in corpus[:5]:
        print(stems)

['comenc', 'a', 'trabaj', 'y', 'me', 'peg', 'me', 'maltrat', 'con', 'chicot']
['mis', 'patron', 'me', 'peg', 'porqu', 'no', 'me', 'quer', 'apur', 'porqu', 'era', 'floj']
['por', 'eso', 'me', 'hab', 'peg']
['cuand', 'me', 'peg', 'ya', 'entonc', 'me', 'quit']
['pues', 'entonc', 'no', 'quis', 'trabaj']


### Paso 2

Insertar símbolos de inicio y final de cadena.

In [5]:
corpus = list(map(lambda stems: ['<BOS>', *stems, '<EOS>'], corpus))
for stems in corpus[:5]:
    print(stems)

['<BOS>', 'comenc', 'a', 'trabaj', 'y', 'me', 'peg', 'me', 'maltrat', 'con', 'chicot', '<EOS>']
['<BOS>', 'mis', 'patron', 'me', 'peg', 'porqu', 'no', 'me', 'quer', 'apur', 'porqu', 'era', 'floj', '<EOS>']
['<BOS>', 'por', 'eso', 'me', 'hab', 'peg', '<EOS>']
['<BOS>', 'cuand', 'me', 'peg', 'ya', 'entonc', 'me', 'quit', '<EOS>']
['<BOS>', 'pues', 'entonc', 'no', 'quis', 'trabaj', '<EOS>']


### Paso 3

Obtener los bigramas que aparecen en el texto (indexar numéricamente).